### Импорты


In [1]:
import pandas as pd
import copy

### Класс автобуса

In [2]:
class Bus():
    def __init__(self,busID, free = True, direction = True):
        self.busID = busID
        self.free = free
        #true - forward, false - back
        self.direction = direction
        self.timefree = 21 #&???

    def change_free(self):
        self.free = not(self.free)

    def change_dir(self):
        self.direction = not(self.direction)
        
    def set_time(self, time):
        self.timefree = time
        
    




In [87]:
class Station():
    def __init__(self, timestart, timestop, direction):
        self.timestart = timestart
        self.timestop = timestop
        self.direction = direction
    def print_info(self):
        return self.timestart, self.timestop, self.direction

### Класс расписания


In [ ]:
'''class Timetable():
    def __init__(self):
'''        

In [75]:
df_timetable = pd.read_excel('test.xlsx')
print(df_timetable.head(50))

     time  countf      delayf  countb      delayb  sumf  onef  sumb  oneb
0   03:00       0  нет рейсов       0  нет рейсов  0.00  0.00  0.00  0.00
1   03:30       0  нет рейсов       0  нет рейсов  0.32  0.32  0.30  0.30
2   04:00       0  нет рейсов       0  нет рейсов  0.60  0.28  0.68  0.38
3   04:30       0  нет рейсов       0  нет рейсов  1.17  0.57  1.12  0.44
4   05:00       1          20       0  нет рейсов  1.68  0.51  1.50  0.38
5   05:30       2          20       1          20  2.03  0.35  1.77  0.27
6   06:00       3           9       3          20  2.43  0.41  2.03  0.26
7   06:30       4           9       3           9  3.96  1.53  2.39  0.36
8   07:00       3           8       3           9  4.32  0.36  3.34  0.95
9   07:30       4           8       4           8  4.84  0.51  3.67  0.33
10  08:00       4           8       4           8  5.22  0.38  4.07  0.40
11  08:30       4           8       4           8  5.46  0.24  4.50  0.43
12  09:00       3           8       3 

In [76]:
#Переведем время в datetime
df_timetable["time_dt"] = df_timetable["time"].apply(lambda x: pd.to_datetime(x, format= "%H:%M"))
print(df_timetable["time_dt"].head(5))


0   1900-01-01 03:00:00
1   1900-01-01 03:30:00
2   1900-01-01 04:00:00
3   1900-01-01 04:30:00
4   1900-01-01 05:00:00
Name: time_dt, dtype: datetime64[ns]


In [77]:
#Заменим нет рейсов на 0
df_timetable.loc[df_timetable["delayf"] == "нет рейсов", "delayf"] = 0
df_timetable.loc[df_timetable["delayb"] == "нет рейсов", "delayb"] = 0
df_timetable.head(2)

,time,countf,delayf,countb,delayb,sumf,onef,sumb,oneb,time_dt
0,03:00,0,0,0,0,0.00,0.00,0.0,0.0,1900-01-01 03:00:00
1,03:30,0,0,0,0,0.32,0.32,0.3,0.3,1900-01-01 03:30:00


### MainTest


In [55]:
# Расчитаем среднее время проезда в обоих направлениях
#Средняя скорость автобуса 19 км/ч
avetimef = round(6.48/19*60, 2)
avetimeb = 5.63/19*60
avtfm = avetimef // 1
avtfs = avetimef % 1 * 60  // 1
avtbm = avetimeb // 1
avtbs = avetimeb % 1 * 60  // 1
print(avtfm, avtfs, avtbm, avtbs)

20.0 27.0 17.0 46.0


In [13]:
#создаем тайминги
ftime = pd.to_datetime(df_timetable["time"][0], format= "%H:%M")
print(ftime)
inter = [ftime + pd.Timedelta(hours = i, minutes=j) for i in range(24) for j in range(5,60,5)]
print(inter[0])

1900-01-01 03:00:00
1900-01-01 03:05:00


In [98]:
lastsend1 = pd.to_datetime("0:0", format= "%H:%M")
test = pd.to_datetime("2:0", format= "%H:%M")

print(lastsend1 < test)

True


In [132]:
dirtt = {} #словарь со списками точек входа/выхода для каждого автобуса

freebus = [] #свободные
busybus = [] #занятые
for i in range(8):
    freebus.append(Bus(i+1))
    dirtt[i+1] = []
buspark = copy.copy(freebus) #парк автобусов
print(dirtt)

{1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}


In [133]:
ftime = pd.to_datetime(df_timetable["time"][0], format= "%H:%M")

nowbusf = 0
nowbusb = 0

lastsendf = pd.to_datetime("0:0", format= "%H:%M")
lastsendb = pd.to_datetime("0:0", format= "%H:%M")
for i in range(24):
    for j in range(0, 60, 5):
        ntime = ftime + pd.Timedelta(hours = i, minutes=j)
        if ntime.day == 2:
            ntime -= pd.Timedelta(days = 1)
        if df_timetable["time_dt"].isin([ntime]).any():
            row = df_timetable.loc[df_timetable["time_dt"] == ntime]
            cbf = int(row["countf"])
            cbb = int(row["countb"])
            delf = int(row["delayf"])
            delb = int(row["delayb"])
        
        deltasendf = (test - lastsendf).seconds/60
        deltasendb = (test - lastsendf).seconds/60
        
        if ((nowbusf + nowbusb) >= (cbf + cbb)) and ((cbf + cbb) != 0):
            for j in buspark:
                if not(j.free):
                    if dirtt[j.busID][-1].timestop < ntime:
                        j.change_free()
                        if j.direction:
                            nowbusf -= 1
                        else: 
                            nowbusb -= 1
                        j.change_dir()
        
        if ((nowbusf + nowbusb) != (cbf + cbb)):
            #переделать в отдельную функцию
            for j in buspark:
                if j.free:
                    if nowbusf != cbf and j.direction and (deltasendf - delf) >= 0:
                        stoptime = ntime + pd.Timedelta(minutes = avtfm, seconds = avtfs)
                        st = Station(ntime, stoptime, True)
                        dirtt[j.busID].append(st)
                        lastsendf = ntime
                        j.change_free()
                        nowbusf += 1
                    elif nowbusb != cbb and not(j.direction) and (deltasendb - delb) >= 0:
                        stoptime = ntime + pd.Timedelta(minutes = avtfm, seconds = avtfs)
                        st = Station(ntime, stoptime, False)
                        dirtt[j.busID].append(st)
                        lastsendb = ntime
                        j.change_free()
                        nowbusb += 1
                
            
        print(ntime, nowbusf, nowbusb, cbf, cbb)
                        
                    
                    
# print(dirtt)
                        
            
            
            

            

1900-01-01 03:00:00 0 0 0 0
1900-01-01 03:05:00 0 0 0 0
1900-01-01 03:10:00 0 0 0 0
1900-01-01 03:15:00 0 0 0 0
1900-01-01 03:20:00 0 0 0 0
1900-01-01 03:25:00 0 0 0 0
1900-01-01 03:30:00 0 0 0 0
1900-01-01 03:35:00 0 0 0 0
1900-01-01 03:40:00 0 0 0 0
1900-01-01 03:45:00 0 0 0 0
1900-01-01 03:50:00 0 0 0 0
1900-01-01 03:55:00 0 0 0 0
1900-01-01 04:00:00 0 0 0 0
1900-01-01 04:05:00 0 0 0 0
1900-01-01 04:10:00 0 0 0 0
1900-01-01 04:15:00 0 0 0 0
1900-01-01 04:20:00 0 0 0 0
1900-01-01 04:25:00 0 0 0 0
1900-01-01 04:30:00 0 0 0 0
1900-01-01 04:35:00 0 0 0 0
1900-01-01 04:40:00 0 0 0 0
1900-01-01 04:45:00 0 0 0 0
1900-01-01 04:50:00 0 0 0 0
1900-01-01 04:55:00 0 0 0 0
1900-01-01 05:00:00 1 0 1 0
1900-01-01 05:05:00 1 0 1 0
1900-01-01 05:10:00 1 0 1 0
1900-01-01 05:15:00 1 0 1 0
1900-01-01 05:20:00 1 0 1 0
1900-01-01 05:25:00 1 0 1 0
1900-01-01 05:30:00 2 1 2 1
1900-01-01 05:35:00 2 1 2 1
1900-01-01 05:40:00 2 1 2 1
1900-01-01 05:45:00 2 1 2 1
1900-01-01 05:50:00 2 1 2 1
1900-01-01 05:55:00 

In [134]:
test2 = {}
for k,v in dirtt.items():
    test2[k] = []
    for i in dirtt[k]:
        test2[k].append(i.print_info())
        print(k, i.print_info())

1 (Timestamp('1900-01-01 05:00:00'), Timestamp('1900-01-01 05:20:27'), True)
1 (Timestamp('1900-01-01 05:30:00'), Timestamp('1900-01-01 05:50:27'), False)
1 (Timestamp('1900-01-01 05:55:00'), Timestamp('1900-01-01 06:15:27'), True)
1 (Timestamp('1900-01-01 07:00:00'), Timestamp('1900-01-01 07:20:27'), False)
1 (Timestamp('1900-01-01 07:25:00'), Timestamp('1900-01-01 07:45:27'), True)
1 (Timestamp('1900-01-01 07:50:00'), Timestamp('1900-01-01 08:10:27'), False)
1 (Timestamp('1900-01-01 08:15:00'), Timestamp('1900-01-01 08:35:27'), True)
1 (Timestamp('1900-01-01 08:40:00'), Timestamp('1900-01-01 09:00:27'), False)
1 (Timestamp('1900-01-01 09:05:00'), Timestamp('1900-01-01 09:25:27'), True)
1 (Timestamp('1900-01-01 09:30:00'), Timestamp('1900-01-01 09:50:27'), False)
1 (Timestamp('1900-01-01 09:55:00'), Timestamp('1900-01-01 10:15:27'), True)
1 (Timestamp('1900-01-01 10:20:00'), Timestamp('1900-01-01 10:40:27'), False)
1 (Timestamp('1900-01-01 10:45:00'), Timestamp('1900-01-01 11:05:27'),

In [127]:
df2 = pd.DataFrame(list(test2.items()), columns=['busid', 'dots'])
df2.head(10)

,busid,dots
0,1,"[(1900-01-01 05:00:00, 1900-01-01 05:20:27, Tr..."
1,2,"[(1900-01-01 05:25:00, 1900-01-01 05:45:27, Tr..."
2,3,"[(1900-01-01 05:30:00, 1900-01-01 05:50:27, Tr..."
3,4,"[(1900-01-01 05:50:00, 1900-01-01 06:10:27, Tr..."
4,5,"[(1900-01-01 06:00:00, 1900-01-01 06:20:27, Tr..."
5,6,"[(1900-01-01 06:30:00, 1900-01-01 06:50:27, Tr..."
6,7,"[(1900-01-01 07:00:00, 1900-01-01 07:20:27, Tr..."
7,8,"[(1900-01-01 07:30:00, 1900-01-01 07:50:27, Tr..."


In [128]:
df2.to_excel('test3.xlsx')